# General

In [ ]:
# pd.set_option('display.max_colwidth', 1000)

In [ ]:
# !pip install nltk

In [ ]:
# import nltk
# nltk.download('wordnet')
# nltk.download('omw-1.4')

# from nltk.stem import WordNetLemmatizer
# lemmatizer = WordNetLemmatizer()

# Kaggle Dataset

In [2]:
import pandas as pd

df = pd.read_csv('./data/artists.csv')
print(df.shape)
df.head()

(1466083, 10)


/var/folders/39/yqq6m0qn4rj4_j3d6xk0bv_h0000gn/T/ipykernel_17358/926555680.py:3: DtypeWarning: Columns (2,4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./data/artists.csv')


,mbid,artist_mb,artist_lastfm,country_mb,country_lastfm,tags_mb,tags_lastfm,listeners_lastfm,scrobbles_lastfm,ambiguous_artist
0,cc197bad-dc9c-440d-a5b5-d52ba2e14234,Coldplay,Coldplay,United Kingdom,United Kingdom,rock; pop; alternative rock; british; uk; brit...,rock; alternative; britpop; alternative rock; ...,5381567.0,360111850.0,False
1,a74b1b7f-71a5-4011-9441-d0b5e4122711,Radiohead,Radiohead,United Kingdom,United Kingdom,rock; electronic; alternative rock; british; g...,alternative; alternative rock; rock; indie; el...,4732528.0,499548797.0,False
2,8bfac288-ccc5-448d-9573-c33ea2aa5c30,Red Hot Chili Peppers,Red Hot Chili Peppers,United States,United States,rock; alternative rock; 80s; 90s; rap; metal; ...,rock; alternative rock; alternative; Funk Rock...,4620835.0,293784041.0,False
3,73e5e69d-3554-40d8-8516-00cb38737a1c,Rihanna,Rihanna,United States,Barbados; United States,pop; dance; hip hop; reggae; contemporary r b;...,pop; rnb; female vocalists; dance; Hip-Hop; Ri...,4558193.0,199248986.0,False
4,b95ce3ff-3d05-4e87-9e01-c97b66af13d4,Eminem,Eminem,United States,United States,turkish; rap; american; hip-hop; hip hop; hiph...,rap; Hip-Hop; Eminem; hip hop; pop; american; ...,4517997.0,199507511.0,False


In [3]:
allTagList = []

def clean(tags):
    tagList = tags.split(" ")
    tagSet = set()
    for tag in tagList:
        if tag != "" and len(tag) >= 3 and len(tag) <= 20:
            tagSet.add(tag)
            allTagList.append(tag)
    
    return " ".join(tagSet)

# get non-ambiguous artists
df2 = df[df['ambiguous_artist'] == False]

# get artist name and tags only
df2 = df2[['tags_mb', 'artist_mb']]

# remove duplicate artists
df2['artist_mb_duplicate'] = df2.loc[:, 'artist_mb']
df2['artist_mb_duplicate'] = df2['artist_mb_duplicate'].str.lower()
df2 = df2.drop_duplicates(subset=['artist_mb_duplicate'])


df2['tags_mb'] = df2['tags_mb'].str.lower()
df2 = df2.dropna(subset=['tags_mb'])
df2 = df2.drop_duplicates(subset=['artist_mb'])
df2 = df2[df2['tags_mb'].apply(lambda x: isinstance(x, str))]

# make each tag a single word
df2['tags_mb'] = df2['tags_mb'].str.replace(' ', '')
df2['tags_mb'] = df2['tags_mb'].str.replace(';', ' ')

# remove special characters
df2['tags_mb'] = df2['tags_mb'].str.replace(
    r'[^a-z0-9\s]',
    '',
    regex=True,
)

# remove tag duplicates and clean tags
df2['tags_mb'] = df2['tags_mb'].apply(clean)

allTagList.sort()
print(allTagList)

print(df2.shape)
df2.head()

['002894790056', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '00s', '0100', '028943639521', '100', '100', '100', '100', '1000apesinaroom', '1000daysofrain', '100recordz', '100th', '1014', '1033', '10s', '10s', '10s', '10s', '10s', '10s', '10s', '10s', '10s', '10s', '10s', '10s', '10s', '10s', '10s', '10s', '10s', '10s', '10s', '10s', '10s', '10s', '10s', '10s', '10s', '10s', '10s', '10s', '10s', '10s', '10s', '10s', '10s', '10s', '

,tags_mb,artist_mb,artist_mb_duplicate
0,alternativerock pop english poprock ambientpop...,Coldplay,coldplay
1,grunge english british bootleg sacredcows nude...,Radiohead,radiohead
2,poprock american funkrock crossover 90s funk 0...,Red Hot Chili Peppers,red hot chili peppers
3,pop electrohouse barbadian electropop dance ri...,Rihanna,rihanna
4,poprap 8mile popandchart rapper american midwe...,Eminem,eminem


In [4]:
df2 = df2[:50000]

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
vectorized = vectorizer.fit_transform(df2['tags_mb'])

In [6]:
from sklearn.metrics.pairwise import cosine_similarity

similarities = cosine_similarity(vectorized)

In [7]:
df3 = pd.DataFrame(similarities, columns=df2['artist_mb'], index=df2['artist_mb']).reset_index()

df3.head()

artist_mb,artist_mb,Coldplay,Radiohead,Red Hot Chili Peppers,Rihanna,Eminem,The Killers,Kanye West,Nirvana,Muse,...,Irrsterne,Irwin Gage,Julián Aguirre,La Gente de Omar Geles,Les Apaches,MASTERLINK,Mörder Machine,Out of Darkness,Rat Scabies,Richard Williams
0,Coldplay,1.000000,0.305068,0.068752,0.018188,0.000000,0.115676,0.028909,0.061714,0.215478,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,Radiohead,0.305068,1.000000,0.032766,0.000000,0.000000,0.061752,0.000000,0.097302,0.171929,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.111955,0.0
2,Red Hot Chili Peppers,0.068752,0.032766,1.000000,0.034782,0.089128,0.167616,0.027038,0.150034,0.164141,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,Rihanna,0.018188,0.000000,0.034782,1.000000,0.047933,0.000000,0.100217,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,Eminem,0.000000,0.000000,0.089128,0.047933,1.000000,0.025704,0.277201,0.017342,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [8]:
input_artist = "Coldplay"
recommendations = pd.DataFrame(df3.nlargest(11,input_artist)['artist_mb'])
recommendations = recommendations[recommendations['artist_mb']!=input_artist]
print(recommendations)

                          artist_mb
98                            Keane
912                      Starsailor
1403  The Good, the Bad & the Queen
738                 Vanessa Carlton
723                       Ben Folds
3526                  Jarvis Cocker
71                          Placebo
3578                      Tom McRae
18                            Oasis
210                           Train


# Scraped Musicbrainz 2023 Dataset

In [10]:
import pandas as pd

df_mb = pd.read_csv('./data/mb_export.csv')
print(df_mb.shape)
df_mb.head()

(2252446, 5)


,ArtistId,mbid,artist,AreaId,tag
0,1,89ad4ac3-39f7-470e-963a-56509c546377,Various Artists,NaN,rock; trance; electronic; classical; pop; spec...
1,2,c06aa285-520e-40c0-b776-83d2c9e8a6d1,Deleted Artist,NaN,NaN
2,4,10adbe5e-a2c0-4bf3-8249-2b4cbf6e6ca8,Massive Attack,3821.0,trip-hop; rock; electronic; downtempo; alterna...
3,6,ea738cc5-5b1a-44a4-94ab-ed0c1bd71ecf,Apartment 26,221.0,electronic; jazz; metal; british; uk; nu metal...
4,7,025d1400-5420-4fc8-a482-2b59096fd22c,Dr. Evil,NaN,bogus artist; fictitious artist; fictitious-ar...


In [11]:
df_mb2 = df_mb.dropna()
df_mb2.shape

(155500, 5)

In [12]:
df_mb2 = df_mb2[['artist', 'tag']]

allTagList = []

def clean(tags):
    tagList = tags.split(" ")
    tagSet = set()
    for tag in tagList:
        if tag != "" and len(tag) >= 3 and len(tag) <= 20:
            tagSet.add(tag)
            allTagList.append(tag)
    
    return " ".join(tagSet)

# remove duplicate artists
df_mb2['artist'] = df_mb2['artist'].str.lower()
df_mb2 = df_mb2.drop_duplicates(subset=['artist'])

# make each tag a single word
df_mb2['tag'] = df_mb2['tag'].str.replace(' ', '')
df_mb2['tag'] = df_mb2['tag'].str.replace(';', ' ')

# remove special characters
df_mb2['tag'] = df_mb2['tag'].str.replace(
    r'[^a-z0-9\s]',
    '',
    regex=True,
)

# remove tag duplicates and clean tags
df_mb2['tag'] = df_mb2['tag'].apply(clean)

print(df_mb2.shape)
df_mb2.head()


(148783, 2)


,artist,tag
2,massive attack,downtempo alternativerock triphop english dub ...
3,apartment 26,numetal english jazzmetal metal british electr...
5,robert miles,downtempo house triphop trance progressivehous...
6,vincent gallo,warp american
7,squirrel nut zippers,swingrevival swing american jazz


In [13]:
df_mb2 = df_mb2[:50000]

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
vectorized = vectorizer.fit_transform(df_mb2['tag'])

In [15]:
from sklearn.metrics.pairwise import cosine_similarity

similarities = cosine_similarity(vectorized)

In [16]:
df_mb3 = pd.DataFrame(similarities, columns=df_mb2['artist'], index=df_mb2['artist']).reset_index()

df_mb3.head()

In [ ]:
input_artist = "Coldplay"
recommendations = pd.DataFrame(df_mb3.nlargest(11,input_artist)['artist'])
recommendations = recommendations[recommendations['artist']!=input_artist]
print(recommendations)

                          artist_mb
98                            Keane
912                      Starsailor
1403  The Good, the Bad & the Queen
738                 Vanessa Carlton
3526                  Jarvis Cocker
723                       Ben Folds
71                          Placebo
3578                      Tom McRae
210                           Train
18                            Oasis
